In [ ]:
!pip install -r requirements.txt

In [1]:
from data.create_dataloaders import create_dataloaders
from model.smooth_cross_entropy import smooth_crossentropy
from model.shake_pyramidnet import ShakePyramidNet
from model.pyramidnet import PyramidNet
from model.wide_res_net import WideResNet
from train import create_model_fun, OptimizerType, train_multiple_models
import torch

batch_size = 128
threads = 4

depth = 16
width_factor = 8
dropout = 0.0

device = torch.device(
    "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
model_fun_WRN = create_model_fun(WideResNet, depth=depth, width_factor=width_factor, dropout=dropout, in_channels=3, labels=100, device=device)

dataset_name = 'cifar10'

model_fun_pyramidnet = create_model_fun(PyramidNet,device=device, dataset=dataset_name, depth=110, alpha=48, num_classes=10, bottleneck=False)

model_fun_shake_pyramidnet = create_model_fun(ShakePyramidNet, device=device, depth=110,
                                              alpha=48, num_classes=10)

train_dataloader, val_dataloader, test_dataloader = create_dataloaders(dataset_name=dataset_name,
                                                                       batch_size=batch_size,
                                                                       num_workers=threads,
                                                                       validation_split=0.0)

N = 3

num_epochs = 400

configs = [
    # CIFAR 10
    {"model": model_fun_WRN, "criterion": smooth_crossentropy,
     "optimizer": {"optimizer_type": OptimizerType.SGD, "learning_rate": 0.1},
     "num_epochs": num_epochs, "model_name": "WRN", "name_suffix": ""},

    {"model": model_fun_WRN, "criterion": smooth_crossentropy,
     "optimizer": {"optimizer_type": OptimizerType.SAM, "learning_rate": 0.1, "momentum": 0.9, "weight_decay": 5e-4,
                   "rho": 0.05},
     "num_epochs": int(num_epochs / 2), "model_name": "WRN", "name_suffix": ""},


    {"model": model_fun_pyramidnet, "criterion": smooth_crossentropy,
    "optimizer": {"optimizer_type": OptimizerType.SGD, "learning_rate": 0.05, "momentum": 0.9, "weight_decay": 5e-4},
    "num_epochs": num_epochs, "model_name": "PyramidNet-ShakeDrop", "name_suffix": ""},

    {"model": model_fun_pyramidnet, "criterion": smooth_crossentropy,
    "optimizer": {"optimizer_type": OptimizerType.SAM, "learning_rate": 0.05, "momentum": 0.9, "weight_decay": 5e-4,
                "rho": 0.05},
    "num_epochs": int(num_epochs / 2), "model_name": "PyramidNet-ShakeDrop", "name_suffix": ""},
]

repeated_configs = []

for i in range(N):
    for config in configs:
        new_config = config
        new_config["name_suffix"] = f"{i + 1}"
        repeated_configs.append(new_config)


train_multiple_models(repeated_configs, train_dataloader, val_dataloader, test_dataloader, dataset_name, device=device)

wandb: Currently logged in as: jhostic (jhostic-fer) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin



Training model: WRN-SGD

Epoch 1/2


Training:  72%|███████▏  | 252/352 [02:56<01:09,  1.43it/s, batch_loss=1.5398, batch_acc=0.2578, avg_loss=1.6814]


KeyboardInterrupt: 

In [ ]:
dataset_name = 'cifar100'

model_fun_WRN = create_model_fun(WideResNet, depth=depth, width_factor=width_factor, dropout=dropout, in_channels=3, labels=100, device=device)
model_fun_pyramidnet = create_model_fun(PyramidNet, device=device, dataset=dataset_name, depth=110, alpha=48, num_classes=100, bottleneck=False)
model_fun_shake_pyramidnet = create_model_fun(ShakePyramidNet, device=device, depth=110,
                                              alpha=48, num_classes=100)
train_dataloader, val_dataloader, test_dataloader = create_dataloaders(dataset_name=dataset_name,
                                                                       batch_size=batch_size,
                                                                       num_workers=threads,
                                                                       validation_split=0.0)

configs = [
    {"model": model_fun_WRN, "criterion": smooth_crossentropy,
     "optimizer": {"optimizer_type": OptimizerType.SGD, "learning_rate": 0.1},
     "num_epochs": num_epochs, "model_name": "WRN", "name_suffix": ""},
    {"model": model_fun_WRN, "criterion": smooth_crossentropy,
     "optimizer": {"optimizer_type": OptimizerType.SAM, "learning_rate": 0.1, "momentum": 0.9, "weight_decay": 5e-4,
                   "rho": 0.1},
     "num_epochs": int(num_epochs / 2), "model_name": "WRN", "name_suffix": ""},

    {"model": model_fun_pyramidnet, "criterion": smooth_crossentropy,
     "optimizer": {"optimizer_type": OptimizerType.SGD, "learning_rate": 0.05, "momentum": 0.9,
                   "weight_decay": 5e-4},
     "num_epochs": num_epochs, "model_name": "PyramidNet-ShakeDrop", "name_suffix": ""},
    {"model": model_fun_pyramidnet, "criterion": smooth_crossentropy,
     "optimizer": {"optimizer_type": OptimizerType.SAM, "learning_rate": 0.05, "momentum": 0.9,
                   "weight_decay": 5e-4,
                   "rho": 0.2},
     "num_epochs": int(num_epochs / 2), "model_name": "PyramidNet-ShakeDrop", "name_suffix": ""},

]

train_multiple_models(configs, train_dataloader, val_dataloader, test_dataloader, dataset_name, device=device)